In [1]:
import pandas as pd
import numpy as np
import json

recipe_file_path = './server/dp/datasets/epi_r.csv'
recipe_json_file_path = "./server/dp/datasets/full_format_recipes.json"


In [2]:
def m1():
    recipe_json_1 = pd.read_json(recipe_json_file_path)
    # res = recipe_json_1.iloc[0:1]['categories']
    gen = recipe_json_1.iterrows()
    # for index in range(0,10):
    #     next(gen)
    for index in range(0,10):
        _, res = next(gen)
        res = res['categories']
        res = np.squeeze(res)
        print(res)
        print(type(res))
        res = [x.lower() for x in res]
        res = sorted(res)
        yield res


In [4]:
recipe_df_1 = pd.read_csv(recipe_file_path)

In [5]:
# pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_rows', 10)
# print(recipe_df_1.iloc[0:1])
# for index, recipe in recipe_df_1.iterrows():
def m2():
    for i in range(0,10):
        is_one_hot_true = map(lambda x: x == np.float(1), recipe_df_1.iloc[i])
        # Recipe with only one-hot encoding column.
        one_hot_recipe = recipe_df_1.iloc[i][is_one_hot_true]
        res2 = sorted(one_hot_recipe.index.values.tolist())
        yield res2


In [6]:
m1 = m1()
m2 = m2()
for i in range(0,10):
    print(next(m1) == next(m2))


['Sandwich' 'Bean' 'Fruit' 'Tomato' 'turkey' 'Vegetable' 'Kid-Friendly'
 'Apple' 'Lentil' 'Lettuce' 'Cookie']
<class 'numpy.ndarray'>
True
['Food Processor' 'Onion' 'Pork' 'Bake' 'Bastille Day' "New Year's Eve"
 'Dried Fruit' 'Port' 'Winter' 'Chill' 'Bon Appétit']
<class 'numpy.ndarray'>
True
['Soup/Stew' 'Dairy' 'Potato' 'Vegetable' 'Fennel' 'Gourmet' 'New York']
<class 'numpy.ndarray'>
True
['Fish' 'Olive' 'Tomato' 'Sauté' 'Low Fat' 'Low Cal' 'High Fiber' 'Dinner'
 'Healthy' 'Simmer' 'Bon Appétit' 'Pescatarian' 'Dairy Free' 'Peanut Free'
 'Tree Nut Free' 'Soy Free' 'Kosher']
<class 'numpy.ndarray'>
True
['Cheese' 'Dairy' 'Pasta' 'Vegetable' 'Side' 'Bake' 'Vegetarian'
 'Quick & Easy' 'Fall' 'Bon Appétit' 'California']
<class 'numpy.ndarray'>
True
['Sandwich' 'Food Processor' 'Tomato' 'Kid-Friendly' 'Quick & Easy'
 'Basil' 'Bacon' 'Summer' 'Lettuce' 'Bon Appétit']
<class 'numpy.ndarray'>
True
['Salad' 'Potato' 'Side' 'Easter' 'Low Fat' 'Quick & Easy' 'Ham'
 'Asparagus' 'Spinach' 'Carro

In [18]:
# This sort function might not be necessary since it can be achieved by directly calling df.sort_values().
def find_matching_recipes(keywords, result_count=15):
    """
    Given a list of keywords find recipes that match the keywords and return all the matched recipes
    and their matching percentage.
    Args:
        keywords: A 'list' containing all the ingredients and other keywords in user's inventory and
         other keywords such as allergy info.
    Returns:
        partial_result_json: A list of dictionary containing result json objects.
        partial_result_df: A Pandas DataFrame containing:
            Title: The name of the recipe with at least one keyword found in it.
            Match Found: The number of keywords found in the recipe given a querying list of keywords.
            Match Percentage: The percentage of matching, defined as match found divided by item count in each recipe.
            Match Items: A list of matched keywords found in each recipe.
    """
    # Read recipes information from epi.csv

    recipe_json_1 = pd.read_json(recipe_json_file_path)
    # res = recipe_json_1.iloc[0:1]['categories']
    # for index in range(0,10):
    #     next(gen)

    result_list = []
    for index, recipe in recipe_json_1.iterrows():
        # The recipe title to be used as a key in found_count for storing recipe result name.
        recipe_name = recipe["title"]

        recipe_ingredients = recipe['categories']
        recipe_ingredients = np.squeeze(recipe_ingredients)
        recipe_ingredients.reshape((len(recipe_ingredients),1))
        recipe_ingredients = [x.lower() for x in recipe_ingredients]
        # res = sorted(res)

        found = 0
        found_keywords = []
        # Loop and count matching keywords in a recipe.
        for keyword in keywords:
            if keyword in recipe_ingredients:
                found += 1
                found_keywords.append(keyword)
        if found != 0:
            # matching_percentage = found_count[recipe_name] / one_hot_recipe.count() * 100
            matching_percentage = found / len(recipe_ingredients) * 100.00

            # Add results to DataFrame.
            result_list.append({'Title': recipe_name,
                                'Match Found': found,
                                'Match Percentage': matching_percentage,
                                'Match Items': found_keywords})

    # Convert list of dicts to DataFrame.
    result_df = pd.DataFrame(result_list)

    # Specify DataFrame desired column order.
    desired_order = ['Title', 'Match Found', 'Match Percentage', 'Match Items']
    result_df = result_df[desired_order]

    # Sort the result DataFrame based on the number of item found in descending order.
    result_df = result_df.sort_values('Match Found', ascending=False)

    # Dataframe containing only the number of result desired.
    partial_result_df = result_df[0:result_count]

    # Get the index number of the matched recipe from the partial result.
    result_indexes = partial_result_df.index.values.tolist()

    # Read recipe.json file
    recipes_json_df = pd.read_json(recipe_json_file_path)

    # Get the partial recipe.json, all row in the indexes, and all col in the dataframe.
    partial_recipes_json_df = recipes_json_df.iloc[result_indexes, :]

    # Convert Dataframe to JSON
    partial_result_json = partial_recipes_json_df.to_json(orient='index')
    partial_result_json = json.JSONDecoder().decode(partial_result_json)

    return partial_result_json, partial_result_df


In [19]:
import time
tic = time.time()
result = find_matching_recipes(keywords_from_inventory)
toc = time.time()
print(str(toc-tic) + " seconds")


TypeError: len() of unsized object

In [10]:
# This sort function might not be necessary since it can be achieved by directly calling df.sort_values().
def find_matching_recipes(keywords, result_count=15):
    """
    Given a list of keywords find recipes that match the keywords and return all the matched recipes
    and their matching percentage.
    Args:
        keywords: A 'list' containing all the ingredients and other keywords in user's inventory and
         other keywords such as allergy info.
    Returns:
        partial_result_json: A list of dictionary containing result json objects.
        partial_result_df: A Pandas DataFrame containing:
            Title: The name of the recipe with at least one keyword found in it.
            Match Found: The number of keywords found in the recipe given a querying list of keywords.
            Match Percentage: The percentage of matching, defined as match found divided by item count in each recipe.
            Match Items: A list of matched keywords found in each recipe.
    """
    # Read recipes information from epi.csv
    recipes_df = pd.read_csv(recipe_file_path)

    result_list = []
    for index, recipe in recipes_df.iterrows():
        # The recipe title to be used as a key in found_count for storing recipe result name.
        recipe_name = recipe["title"]

        is_one_hot_true = map(lambda x: x == np.float(1), recipe)

        # Recipe with only one-hot encoding column.
        one_hot_recipe = recipe[is_one_hot_true]

        found = 0
        found_keywords = []
        # Loop and count matching keywords in a recipe.
        for keyword in keywords:
            if keyword in one_hot_recipe:
                found += 1
                found_keywords.append(keyword)
        if found != 0:
            # matching_percentage = found_count[recipe_name] / one_hot_recipe.count() * 100
            matching_percentage = found / one_hot_recipe.count() * 100.00

            # Add results to DataFrame.
            result_list.append({'Title': recipe_name,
                                'Match Found': found,
                                'Match Percentage': matching_percentage,
                                'Match Items': found_keywords})

    # Convert list of dicts to DataFrame.
    result_df = pd.DataFrame(result_list)

    # Specify DataFrame desired column order.
    desired_order = ['Title', 'Match Found', 'Match Percentage', 'Match Items']
    result_df = result_df[desired_order]

    # Sort the result DataFrame based on the number of item found in descending order.
    result_df = result_df.sort_values('Match Found', ascending=False)

    # Dataframe containing only the number of result desired.
    partial_result_df = result_df[0:result_count]

    # Get the index number of the matched recipe from the partial result.
    result_indexes = partial_result_df.index.values.tolist()

    # Read recipe.json file
    recipes_json_df = pd.read_json(recipe_json_file_path)

    # Get the partial recipe.json, all row in the indexes, and all col in the dataframe.
    partial_recipes_json_df = recipes_json_df.iloc[result_indexes, :]

    # Convert Dataframe to JSON
    partial_result_json = partial_recipes_json_df.to_json(orient='index')
    partial_result_json = json.JSONDecoder().decode(partial_result_json)

    return partial_result_json, partial_result_df

In [11]:
keywords_from_inventory = ["lettuce", "chicken", "apple", "tomato", "turkey", "bean"]

In [14]:
import time
tic = time.time()
result = find_matching_recipes(keywords_from_inventory)
toc = time.time()
print(str(toc-tic) + " seconds")


TypeError: iteration over a 0-d array

In [5]:
# Test for find_matching_recipes
recipes_df_json_m = pd.read_json(recipe_json_file_path)
print(list(recipes_df_json_m))

['calories', 'categories', 'date', 'desc', 'directions', 'fat', 'ingredients', 'protein', 'rating', 'sodium', 'title']


In [35]:
result_indexes = result.index.values.tolist()
print(result_indexes)

[0, 1061, 140, 2680, 2392, 694, 365, 859, 4743, 3943, 2070, 3920, 2030, 3140, 2655]


In [36]:
result_json_df = recipes_df_json_m.iloc[result_indexes, :]

In [32]:
from pprint import pprint

result_json = result_json_df.to_json(orient='index')

In [17]:
with open('result.json', 'w') as output_file:
    json.dump(result_json, output_file)

In [66]:
result, result_details = find_matching_recipes(keywords_from_inventory)

In [68]:
print(type(result))
pprint(result)

<class 'dict'>
{'0': {'calories': 426.0,
       'categories': ['Sandwich',
                      'Bean',
                      'Fruit',
                      'Tomato',
                      'turkey',
                      'Vegetable',
                      'Kid-Friendly',
                      'Apple',
                      'Lentil',
                      'Lettuce',
                      'Cookie'],
       'date': 1157083200000,
       'desc': None,
       'directions': ['1. Place the stock, lentils, celery, carrot, thyme, and '
                      'salt in a medium saucepan and bring to a boil. Reduce '
                      'heat to low and simmer until the lentils are tender, '
                      'about 30 minutes, depending on the lentils. (If they '
                      'begin to dry out, add water as needed.) Remove and '
                      'discard the thyme. Drain and transfer the mixture to a '
                      'bowl; let cool.',
                      '2. Fold in 

In [72]:
print("Result details: " + str(type(result_details)))
print(result_details)

Result details: <class 'pandas.core.frame.DataFrame'>
                                                  Title  Match Found  \
0                       Lentil, Apple, and Turkey Wrap             5   
1061              N'awlins Butter Beans with Andouille             4   
140                       New Millennium Waldorf Salad             3   
2680             Turkey Soup with Orzo and White Beans             3   
2392                        Chickpea Soup with Arugula             3   
694                              Two-Bean Turkey Chili             3   
365   Grilled Pork, Tomato, Frisée, and Black Bean S...            3   
859                                Layered Nacho Salad             3   
4743       Baked Chicken with White Beans and Tomatoes             3   
3943                               Fried Chicken Salad             3   
2070  Spicy Chicken Salad with Bell Pepper and Cilan...            3   
3920  Summer Chef's Salad with Grilled Pork, Chicken...            3   
2030  Brai

In [12]:
import time
tic = time.time()
###
toc = time.time()
duration = toc - tic
print(str(duration) + " s")

hello


5.00225830078125 s
